In [11]:
%pip uninstall -y numpy pandas scikit-learn scikit-learn-extra
%pip install numpy==1.24.4 pandas==1.5.3 scikit-learn==1.2.2 scikit-learn-extra==0.2.0


%pip install scikit-learn-extra==0.2.0

import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn_extra.cluster import KMedoids
import numpy as np 

# --- Seu código existente (até a criação da tfidf_matrix e cosine_sim) ---
DATA_PATH = Path(r"C:\Users\melom\Downloads\steam_clean_v2.xlsx") # Ajuste o caminho se necessário
df = pd.read_excel(DATA_PATH)

# Limpeza básica: remover linhas onde 'cbf_text' ou 'name' é NaN, pois são cruciais
df.dropna(subset=['cbf_text', 'name'], inplace=True)
df.reset_index(drop=True, inplace=True) # Resetar o índice após dropna

print(df.shape)
# df.head() # Descomente se quiser ver
# print("Colunas:", df.columns.tolist())
# print("\nTop 10 gêneros:")
# print(df['genres'].value_counts().head(10))
# print("\nAno de lançamento - mínimo, máximo:", df['release_year'].min(), df['release_year'].max())

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000) # Adicionei max_features para controlar a dimensionalidade
tfidf_matrix = vectorizer.fit_transform(df['cbf_text'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) # Isso ainda é útil
indices = pd.Series(df.index, index=df['name']).drop_duplicates()

def get_recommendations_content_based(title, n=10): # Renomeei para clareza
    idx = indices.get(title)
    if idx is None:
        # Tentar encontrar por correspondência parcial se o nome exato não for encontrado
        possible_matches = [name for name in indices.index if title.lower() in name.lower()]
        if not possible_matches:
            return pd.DataFrame({'Aviso': [f'"{title}" não encontrado na base. Nenhuma correspondência parcial.']})
        # Se houver correspondências parciais, pegar a primeira (ou listar para o usuário escolher)
        # Aqui, para simplificar, pegamos a primeira
        print(f"'{title}' não encontrado. Usando a correspondência mais próxima: '{possible_matches[0]}'")
        idx = indices.get(possible_matches[0])
        if idx is None: # Ainda não encontrado (improvável se possible_matches não estiver vazio)
             return pd.DataFrame({'Aviso': [f'"{title}" não encontrado na base.']})

    if isinstance(idx, pd.Series): # Se houver múltiplos jogos com o mesmo nome
        idx = idx.iloc[0] # Pega o primeiro índice

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    game_idx = [i[0] for i in sim_scores]
    return df.loc[game_idx, ['appid','name','genres','price','popularity']]

# --- Implementação do K-Medoids ---

# 1. Escolher o número de clusters (k)
#    Este valor pode ser ajustado e otimizado. Vamos começar com um valor exemplo.
num_clusters = 20 # Exemplo, ajuste conforme necessário

# 2. Instanciar e treinar o modelo KMedoids
#    Usamos 'cosine' como métrica, pois é adequada para dados TF-IDF.
#    'init="k-medoids++"' é uma boa heurística para inicialização.
#    'random_state' para reprodutibilidade.
print(f"\nIniciando K-Medoids com {num_clusters} clusters...")
kmedoids_model = KMedoids(n_clusters=num_clusters, metric='cosine', init='k-medoids++', random_state=42)

# O KMedoids espera que a métrica de 'cosine' seja uma distância, não similaridade.
# A matriz tfidf_matrix já representa os vetores.
# A biblioteca sklearn-extra lida com a distância cosseno corretamente (1 - similaridade).
# No entanto, é comum que KMedoids funcione melhor com matrizes de distância pré-calculadas se houver problemas de escala ou densidade.
# Para TF-IDF, usar 'cosine' diretamente com os vetores da matriz TF-IDF é geralmente aceitável.

# Ajustar o modelo aos dados TF-IDF
# Pode demorar um pouco dependendo do tamanho da tfidf_matrix e do número de clusters
kmedoids_model.fit(tfidf_matrix)
print("K-Medoids treinado.")

# 3. Obter os resultados do clustering
#    'labels_' contém o ID do cluster para cada jogo
df['cluster'] = kmedoids_model.labels_

#    'medoid_indices_' contém os índices dos jogos que são os medoides de cada cluster
medoid_indices = kmedoids_model.medoid_indices_
medoids = df.loc[medoid_indices]

print(f"\nMedoides dos {num_clusters} clusters:")
print(medoids[['name', 'genres', 'cluster']])

# 4. Função de recomendação baseada em K-Medoids
def get_recommendations_kmedoids(title, n=10):
    idx_input_game = indices.get(title)
    
    if idx_input_game is None:
        possible_matches = [name for name in indices.index if title.lower() in name.lower()]
        if not possible_matches:
            return pd.DataFrame({'Aviso': [f'"{title}" não encontrado na base. Nenhuma correspondência parcial.']})
        print(f"'{title}' não encontrado. Usando a correspondência mais próxima: '{possible_matches[0]}'")
        idx_input_game = indices.get(possible_matches[0])
        if idx_input_game is None:
             return pd.DataFrame({'Aviso': [f'"{title}" não encontrado na base.']})

    if isinstance(idx_input_game, pd.Series):
        idx_input_game = idx_input_game.iloc[0]

    # Encontrar o cluster do jogo de entrada
    input_game_cluster = df.loc[idx_input_game, 'cluster']
    
    # Filtrar jogos que estão no mesmo cluster
    games_in_same_cluster = df[df['cluster'] == input_game_cluster]
    
    # Excluir o próprio jogo de entrada da lista de recomendações
    recommendations = games_in_same_cluster[games_in_same_cluster.index != idx_input_game]
    
    # Se houver muitos jogos no cluster, podemos ordená-los por popularidade ou
    # pela similaridade de cosseno com o jogo de entrada (ou com o medoide do cluster).
    # Para este exemplo, vamos apenas pegar os 'n' primeiros ou todos se forem menos que 'n'.
    # Poderíamos também pegar os mais próximos ao medoide do cluster.
    
    # Opcional: Calcular similaridade com o jogo de entrada dentro do cluster
    # Se 'recommendations' for muito grande, pode-se usar a 'cosine_sim' para refinar.
    # Esta etapa pode ser complexa se o cluster for muito grande.
    # Uma abordagem mais simples é pegar amostras aleatórias ou os mais populares.
    
    # Para simplificar, retornamos os n primeiros jogos do cluster (excluindo o de entrada)
    # Poderia ser melhorado ordenando por popularidade ou proximidade ao medoide/jogo de entrada.
    
    # Ordenar por proximidade ao jogo original dentro do cluster
    # Recalcular sim_scores apenas para os jogos do mesmo cluster
    game_indices_in_cluster = recommendations.index
    
    # Verificando se há jogos no cluster além do jogo de entrada
    if game_indices_in_cluster.empty:
        return pd.DataFrame({'Aviso': [f'Não há outros jogos no mesmo cluster para "{title}".']})

    sim_scores_cluster = []
    for game_idx_cluster in game_indices_in_cluster:
        similarity = cosine_sim[idx_input_game, game_idx_cluster]
        sim_scores_cluster.append((game_idx_cluster, similarity))
        
    sim_scores_cluster = sorted(sim_scores_cluster, key=lambda x: x[1], reverse=True)
    
    # Pegar os índices dos n melhores jogos
    recommended_game_indices = [i[0] for i in sim_scores_cluster[:n]]
    
    return df.loc[recommended_game_indices, ['appid','name','genres','price','popularity', 'cluster']]


# --- Exemplos de Uso ---

print("\n--- Recomendações Baseadas em Conteúdo (Similaridade de Cosseno Direta) ---")
game_title_example = 'Counter-Strike' # Garanta que este jogo existe no seu 'name' após a limpeza
recommendations_content = get_recommendations_content_based(game_title_example)
if 'Aviso' in recommendations_content.columns:
    print(recommendations_content['Aviso'].iloc[0])
else:
    print(f"Recomendações para '{game_title_example}':")
    print(recommendations_content)

print("\n--- Recomendações Baseadas em K-Medoids (Mesmo Cluster, Ordenado por Similaridade) ---")
recommendations_kmedoids = get_recommendations_kmedoids(game_title_example)
if 'Aviso' in recommendations_kmedoids.columns:
    print(recommendations_kmedoids['Aviso'].iloc[0])
else:
    print(f"Recomendações do cluster para '{game_title_example}' (Cluster {df.loc[indices.get(game_title_example) if not isinstance(indices.get(game_title_example), pd.Series) else indices.get(game_title_example).iloc[0], 'cluster']}):")
    print(recommendations_kmedoids)

# Mostrar alguns jogos por cluster
print("\n--- Exemplo de Jogos por Cluster ---")
for i in range(min(num_clusters, 3)): # Mostra para os primeiros 3 clusters
    print(f"\nJogos no Cluster {i} (até 5 exemplos):")
    cluster_games = df[df['cluster'] == i][['name', 'genres']].head()
    print(cluster_games)

    # Encontrar o medoide deste cluster
    medoide_idx_cluster_i = kmedoids_model.medoid_indices_[i]
    medoide_name_cluster_i = df.loc[medoide_idx_cluster_i, 'name']
    print(f"Medoide do Cluster {i}: {medoide_name_cluster_i}")

Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached scikit-learn-extra-0.2.0.tar.gz (813 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      Compiling sklearn_extra/utils/_cyfht.pyx because it changed.
      Compiling sklearn_extra/cluster/_k_medoids_helper.pyx because it changed.
      Compiling sklearn_extra/robust/_robust_weighted_estimator_helper.pyx because it changed.
      Compiling sklearn_extra/cluster/_commonnn_inner.pyx because it changed.
      [1/4] Cythonizing sklearn_extra/cluster/_commonnn_inner.pyx
      [2/4] Cythonizing sklearn_extra/cluster/_k_medoids_helper.pyx
      [3/4] Cythonizing sklearn_extra/robust/_robust_weighted_estimator_helper.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
      import sys
      from time import time
      
      from libc.math cimport exp, log, sqrt, pow, fabs
      cimport numpy as np
      from numpy.math cimport INFINITY
      ^
      -------

  Using cached scikit-learn-extra-0.2.0.tar.gz (813 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      Compiling sklearn_extra/utils/_cyfht.pyx because it changed.
      Compiling sklearn_extra/cluster/_k_medoids_helper.pyx because it changed.
      Compiling sklearn_extra/robust/_robust_weighted_estimator_helper.pyx because it changed.
      Compiling sklearn_extra/cluster/_commonnn_inner.pyx because it changed.
      [1/4] Cythonizing sklearn_extra/cluster/_commonnn_inner.pyx
      [2/4] Cythonizing sklearn_extra/cluster/_k_medoids_helper.pyx
      [3/4] Cythonizing sklearn_extra/robust/_robust_weighted_estimator_helper.pyx
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
      import sys
      from time import time
      
      from libc.math cimport exp, log, sqrt, pow, fabs
      cimport numpy as np
      from numpy.math cimport INFINITY
      ^
      -------

ModuleNotFoundError: No module named 'sklearn_extra'

In [ ]:
pip install scikit-learn-extra